# 02 - Quantum Classifier on MNIST (Toy Example)

In [ ]:

import pennylane as qml
from pennylane import numpy as np
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

mnist = fetch_openml('mnist_784', version=1)
X = mnist.data.astype('float32')
y = mnist.target.astype('int64')

mask = (y == 4) | (y == 9)
X, y = X[mask], y[mask]
y = (y == 9).astype(int)

X = StandardScaler().fit_transform(X)
X_pca = PCA(n_components=4).fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=42)


In [ ]:

n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(len(x)):
        qml.RX(x[i], wires=i)

def variational(params):
    for i in range(n_qubits):
        qml.RY(params[i], wires=i)
    for i in range(n_qubits-1):
        qml.CNOT(wires=[i, i+1])

@qml.qnode(dev)
def circuit(x, params):
    feature_map(x)
    variational(params)
    return qml.expval(qml.PauliZ(0))

params = np.random.normal(0, 0.1, size=(n_qubits,), requires_grad=True)


In [ ]:

def cost(params, X, Y):
    preds = [circuit(x, params) for x in X]
    preds = np.sign(preds)
    return np.mean((preds - (2*Y-1))**2)

opt = qml.GradientDescentOptimizer(stepsize=0.1)
for it in range(5):
    params, c = opt.step_and_cost(lambda v: cost(v, X_train[:100], y_train[:100]), params)
    print(f"Iter {it}, Cost {c:.4f}")


In [ ]:

preds = [circuit(x, params) for x in X_test[:100]]
preds = np.sign(preds)
acc = np.mean(preds == (2*y_test[:100]-1))
print("Test Accuracy:", acc)
